### 2 Обработка текста

В предыдущей главе мы углубились в структуру больших языковых моделей (LLM) и узнали, что они предварительно обучаются на огромных объемах текста, обрабатывая
по одному слову (токену) за раз. 

![](images/llm2.1.png)


### 2.1 Векторное представление (vector embedding)

Нейросетевые модели, не могут напрямую обрабатывать текст. Текст как тип данных несовместим с тензорными операциями. Следовательно, нужен способ представлять слова в виде векторов с непрерывными значениями. (ссылка узнать больше в Приложении A, раздел A2.2 «Понимание тензоров».)

Построение векторного представления может быть проведено разными способами (различные модели представления). 
Прочие типы данных также требуют преобразования в тензорный формат.  

![](images/llm2.2.png)

Существует несколько алгоритмов для создания представлений слов. Одним из первых и наиболее популярных примеров является Word2Vec. Основная идея заключается в том, что слова, встречающиеся в схожем контексте, имеют схожее значение. Следовательно, при проецировании в двумерное пространство, можно увидеть, что схожие термины группируются вместе.

![](images/llm2.3.png)

Помимо моделей представления слов, существуют модели представления предложений, абзацев, целых документов.
Таким образом, картинка из предыдущей главы (predicting next world) не вполне корректна. 

Хотя мы можем использовать существующие модели, LLM обычно создают свои собственные представления, которые являются частью входного слоя и обновляются во время обучения. Преимущество такого подхода - оптимизация для конкретной задачи и имеющихся данных. 

Размерность представления (двумерное на рисунке выше) может быть произвольной. Большая размерность может отражать более тонкие отношения, но за счет вычислительной эффективности. Эта размерность называется размерностью скрытых состояний модели (dimensionality of the model's hidden states). 
Эта размерность - компромисс между производительностью и эффективностью. 
GPT-2 (125M параметров) используют размерность 768. 
GPT-3 (175B параметров) использует размерность 12 288.

### 2.2 Токенизация текста

![](images/llm2.4.png)

Текст, который мы будем токенизировать для обучения LLM, представляет собой рассказ Эдит Уортон под названием «Вердикт». Текст доступен в Wikisource по адресу https://en.wikisource.org/wiki/The_Verdict 


In [1]:
# Чтение короткого рассказа в виде образца текста на Python

with open("source/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


Как нам лучше всего разделить этот текст, чтобы получить список токенов? 

Используя простой пример текста, мы можем использовать команду re.split со следующим синтаксисом, чтобы разделить текст на пробельные символы:

In [2]:
import re
text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']



Изменим разделение регулярных выражений на пробелы (\s), запятые и точки ([,.]):

In [3]:
result = re.split(r'([,.]|\s)', text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


Пробелы и прочие специальные символы. При разработке простого токенизатора, следует ли нам кодировать пробелы как отдельные символы или просто удалять их, зависит от нашего приложения и его требований. Удаление пробелов снижает требования к памяти и вычислительным ресурсам. Однако сохранение пробелов может быть полезно, если мы обучаем модели, чувствительные к точной структуре текста (например, код Python, чувствительный к отступам и интервалам). Здесь мы удаляем пробелы для простоты и краткости токенизированных результатов. Позже мы переключимся на схему токенизации, включающую пробелы.

Изменим код, чтобы он мог обрабатывать и другие типы пунктуации, такие как вопросительные знаки, кавычки и двойные тире, которые мы видели ранее в первых 100 символах рассказа Эдит Уортон, а также дополнительные специальные символы.

In [4]:
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


Наша модель токенизации теперь может успешно обрабатывать различные специальные символы в тексте.

![](images/llm2.5.png)

Теперь, когда у нас есть работающий базовый токенизатор, применим его ко всему рассказу Эдит Уортон:

In [5]:
preprocessed = re.split(r'([,.?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))
print(preprocessed[:30])

4649
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


### 2.3 Преобразование токенов в идентификаторы

Это преобразование является промежуточным шагом перед преобразованием в векторное представление.

Чтобы сопоставить ранее сгенерированные токены с идентификаторами, нужно создать словарь. Этот словарь определяет, как мы сопоставляем каждое уникальное слово и специальный символ с уникальным целым числом, как показано на рисунке. Мы создаем словарь, разбивая весь текст обучающего набора данных на отдельные токены. Эти отдельные токены затем сортируются в алфавитном порядке, а повторяющиеся токены удаляются. Затем уникальные токены объединяются в словарь, который определяет сопоставление каждого уникального токена с уникальным целочисленным значением. 

![](images/llm2.6.png)

In [6]:
all_words = sorted(list(set(preprocessed)))
vocab_size = len(all_words)
print(vocab_size)

vocab = {token:integer for integer,token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i > 50:
        break

1159
('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Carlo;', 25)
('Chicago', 26)
('Claude', 27)
('Come', 28)
('Croft', 29)
('Destroyed', 30)
('Devonshire', 31)
('Don', 32)
('Dubarry', 33)
('Emperors', 34)
('Florence', 35)
('For', 36)
('Gallery', 37)
('Gideon', 38)
('Gisburn', 39)
('Gisburns', 40)
('Grafton', 41)
('Greek', 42)
('Grindle', 43)
('Grindle:', 44)
('Grindles', 45)
('HAD', 46)
('Had', 47)
('Hang', 48)
('Has', 49)
('He', 50)
('Her', 51)


Как мы видим, словарь содержит отдельные токены, связанные с уникальными целочисленными метками.
Наша следующая цель — применить этот словарь для преобразования нового текста в идентификаторы токенов, как показано на рисунке.

![](images/llm2.7.png)


Далее, когда мы захотим преобразовать выходные данные LLM из чисел обратно в текст, нам также понадобится способ превратить идентификаторы в текст. Для этого мы можем создать обратную версию словаря, которая сопоставляет идентификаторы с соответствующими текстовыми токенами.

Реализуем полный класс токенизатора и детокенизатора.

In [7]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids]) 
        
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text
    
tokenizer = SimpleTokenizerV1(vocab)
 
text = """"It's the last he painted, you know," Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)

print(ids)
print(tokenizer.decode(ids))

[1, 58, 2, 872, 1013, 615, 541, 763, 5, 1155, 608, 5, 1, 69, 7, 39, 873, 1136, 773, 812, 7]
" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.



### 2.4 Добавление специальных токенов контекста

Изменим токенизатор для обработки неизвестных слов. Также обсудим использование и добавление специальных токенов контекста, которые могут улучшить понимание моделью контекста или другой соответствующей информации в тексте. Эти специальные токены могут включать, например, маркеры для неизвестных слов и границ документа.

В частности, мы изменим словарь и токенизатор для поддержки двух новых токенов: <|unk|> и <|endoftext|>

![](images/llm2.9.png)

Как показано на рисунке, мы можем изменить токенизатор, чтобы он использовал токен <|unk|>, если он встретит слово, которое не является частью словаря. Кроме того, мы добавляем токен между несвязанными текстами. Например, при обучении GPT-подобных LLM на нескольких независимых документах или книгах обычно вставляется токен перед каждым документом или книгой, следующим за предыдущим источником текста Это помогает LLM понять, что, хотя эти текстовые источники объединены для обучения, на самом деле они не связаны друг с другом.

![](images/llm2.10.png)

In [8]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer,token in enumerate(all_tokens)}
 
print(len(vocab.items()))

for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int
                        else "<|unk|>" for item in preprocessed]
 
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
 
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text
    
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)

tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))
print(tokenizer.decode(tokenizer.encode(text)))

1161
('younger', 1156)
('your', 1157)
('yourself', 1158)
('<|endoftext|>', 1159)
('<|unk|>', 1160)
Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.
[1160, 5, 362, 1155, 642, 1000, 10, 1159, 57, 1013, 981, 1009, 738, 1013, 1160, 7]
<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


Выше мы видим, что идентификатор 1159 соответствует токену-разделителю <|endoftext|>, а идентификатор 1160 используются для неизвестных слов.

Сравнивая приведенный выше детокенизированный текст с исходным входным текстом, мы знаем, что набор обучающих данных, рассказ Эдит Уортон «Вердикт», не содержал слов «Привет» и «дворец».

Также рассматривают дополнительные специальные токены, такие как следующие:

[BOS] (начало последовательности): этот токен отмечает начало текста.   
[EOS] (конец последовательности): этот токен располагается в конце текста. Например, при объединении двух разных статей или книг Википедии токен [EOS] указывает, где заканчивается одна статья и начинается следующая.  
[PAD] (заполнение): при обучении LLM с размерами пакетов больше единицы пакет может содержать тексты различной длины. Чтобы гарантировать одинаковую длину всех текстов, более короткие тексты расширяются или «дополняются» с помощью токена [PAD] до длины самого длинного текста в пакете.  

### 2.5 Кодирование пар байтов (Byte pair encoding)

В этом разделе рассматривается более сложная схема токенизации, основанная на концепции, называемой кодированием пар байтов (BPE). Такой токенизатор использовался для обучения LLM, таких как GPT-2, GPT-3 и исходной модели, используемой в ChatGPT.

Поскольку реализация BPE может быть сложной, мы будем использовать существующую библиотеку Python с открытым исходным кодом под названием tiktoken (https://github.com/openai/tiktoken), которая очень эффективно реализует алгоритм BPE на основе исходного кода на Rust. Как и другие библиотеки Python, мы можем установить библиотеку tiktoken через установщик pip Python из терминала.

In [9]:
from importlib.metadata import version
import tiktoken
print("tiktoken version:", version("tiktoken"))

tokenizer = tiktoken.get_encoding("gpt2")

text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace."
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

tiktoken version: 0.6.0
[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]


На основе идентификаторов токенов и декодированного текста выше мы можем сделать два примечательных наблюдения. Во-первых, токену <|endoftext|> присваивается относительно большой идентификатор токена, а именно 50256. Фактически, токенизатор BPE, который использовался для обучения таких моделей, как GPT-2, GPT-3, и исходная модель, используемая в ChatGPT имеет общий размер словаря 50 257, причем <|endoftext|> присвоен самый большой идентификатор токена.

Во-вторых, токенизатор BPE, указанный выше, правильно кодирует и декодирует неизвестные слова, такие как «someunknownPlace». Токенизатор BPE может обрабатывать любое неизвестное слово. Как это достигается без использования токенов <|unk|>?

Алгоритм, лежащий в основе BPE, разбивает слова, которых нет в его предопределенном словаре, на более мелкие подслова или даже отдельные символы, что позволяет ему обрабатывать слова, которых нет в словаре. Таким образом, благодаря алгоритму BPE, если токенизатор встречает незнакомое слово во время токенизации, он может представить его как последовательность токенов или символов подслова, как показано на рисунке.

![](images/llm2.11.png)

Подробное обсуждение и реализация BPE выходит за рамки этой книги, но, если коротко, словарь строится путем итеративного объединения часто встречающихся символов в подслова, а часто встречающихся подслов — в слова. Например, BPE начинает с добавления в свой словарь всех отдельных одиночных символов («a», «b», ...). На следующем этапе он объединяет комбинации символов, которые часто встречаются вместе, в подслова. Например, «d» и «e» могут быть объединены в подслово «de», которое часто встречается во многих английских словах, таких как «define», «dependent», «made» и «hidden».

### 2.6 Выборка данных с помощью скользящего окна

Следующим шагом является создание пар вход-выход, необходимых для обучения LLM.
Как выглядят эти пары вход-выход? 

![](images/llm2.12.png)

Реализуем теперь загрузчик данных, который извлекает пары вход-вызод из набора обучающих данных, используя подход скользящего окна.

Для чего токенизируем весь рассказ The Verdict, с которым мы работали ранее, с помощью токенизатора BPE


In [10]:
with open("source/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
 
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


Один из самых простых и интуитивно понятных способов создания пар входных данных для задачи прогнозирования следующего слова — это создание двух переменных, x и y, где x содержит входные токены, а y содержит целевые значения, которые представляют собой входные данные, сдвинутые на 1:

In [11]:
enc_sample = enc_text[50:]
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


Обрабатывая входные данные вместе с целевыми объектами, которые представляют собой входные данные, сдвинутые на одну позицию, мы можем затем создать задачи прогнозирования следующего слова, следующим образом:

In [12]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


Все, что находится слева от стрелки (---->), относится к входным данным, которые LLM получит, а идентификатор токена в правой части стрелки представляет собой целевой идентификатор токена, который LLM должен предсказать.

В целях иллюстрации давайте повторим предыдущий код, но преобразуем идентификаторы токенов в текст.

In [13]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


Теперь мы создали пары вход-цель, которые можно использовать для обучения LLM в следующих главах.

Прежде чем мы сможем превратить токены во встраивания, есть еще только одна задача, как мы упоминали в начале этой главы: реализация эффективного загрузчика данных, который выполняет итерацию по входному набору данных и возвращает входные и целевые значения в виде тензоров PyTorch, что можно представить себе как как многомерные массивы.

В частности, мы заинтересованы в возврате двух тензоров: входного, содержащего текст, который видит LLM, и целевого тензора, который включает в себя цели, которые LLM должен предсказать, как показано на рисунке 2.13.

Рисунок 2.13. Чтобы реализовать эффективные загрузчики данных, мы собираем входные данные в тензор x, где каждая строка представляет один входной контекст. Второй тензор y содержит соответствующие цели прогнозирования (следующие слова), которые создаются путем сдвига входных данных на одну позицию.

![](images/llm2.13.png)

Хотя на рисунке 2.13 для иллюстрации токены показаны в строковом формате, реализация кода будет работать непосредственно с идентификаторами токенов, поскольку метод кодирования токенизатора BPE выполняет как токенизацию, так и преобразование в идентификаторы токенов за один шаг.

Для эффективной реализации загрузчика данных мы будем использовать встроенные классы PyTorch Dataset и DataLoader. Дополнительную информацию и рекомендации по установке PyTorch см. в разделе A.1.3 «Установка PyTorch» в Приложении A.

Код класса набора данных показан в листинге 2.5:

Листинг 2.5. Набор данных для пакетных входных и целевых значений

In [14]:
import torch
from torch.utils.data import Dataset, DataLoader
 
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.target_ids = []
 
        token_ids = tokenizer.encode(txt)
 
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
 
    def __len__(self):
        return len(self.input_ids)
 
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

Класс GPTDatasetV1 в листинге 2.5 основан на классе набора данных PyTorch и определяет, как отдельные строки извлекаются из набора данных, где каждая строка состоит из нескольких идентификаторов токенов (на основе max_length), назначенных тензору input_chunk. Тензор target_chunk содержит соответствующие цели. Я рекомендую прочитать дальше, чтобы увидеть, как выглядят данные, возвращаемые из этого набора данных, когда мы объединяем набор данных с PyTorch DataLoader — это принесет дополнительную интуицию и ясность.

Если вы впервые знакомы со структурой классов набора данных PyTorch, например, показанной в листинге 2.5, прочтите раздел A.6 «Настройка эффективных загрузчиков данных» в приложении A, в котором объясняется общая структура и использование классов PyTorch Dataset и DataLoader.

Следующий код будет использовать GPTDatasetV1 для пакетной загрузки входных данных через PyTorch DataLoader:

Листинг 2.6. Загрузчик данных для генерации пакетов с парами вход-с

In [15]:
def create_dataloader_v1(txt, batch_size=4, 
        max_length=256, stride=128, shuffle=True, drop_last=True):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last)
    return dataloader

Давайте протестируем загрузчик данных с размером пакета 1 для LLM с размером контекста 4, чтобы понять, как класс GPTDatasetV1 из листинга 2.5 и функция create_dataloader_v1 из листинга 2.6 работают вместе:

In [16]:
with open("source/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
 
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


/tmp/ipykernel_338334/3364176621.py:15: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:84.)
  self.input_ids.append(torch.tensor(input_chunk))


Переменная first_batch содержит два тензора: первый тензор хранит идентификаторы входных токенов, а второй тензор хранит идентификаторы целевых токенов. Поскольку max_length установлено равным 4, каждый из двух тензоров содержит 4 идентификатора токена. Обратите внимание, что входной размер 4 относительно мал и выбран только для иллюстрации. Обычно LLM обучают с размером входных данных не менее 256.

Чтобы проиллюстрировать значение шага=1, давайте возьмем еще один пакет из этого набора данных:

In [17]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


Если мы сравним первый и второй пакет, мы увидим, что идентификаторы токенов второго пакета сдвинуты на одну позицию по сравнению с первым пакетом (например, второй идентификатор во входных данных первого пакета равен 367, что является первым идентификатором вход второго пакета). Настройка шага определяет количество позиций, на которые смещаются входные данные между пакетами, имитируя подход скользящего окна, как показано на рисунке 2.14.

Рисунок 2.14. При создании нескольких пакетов из входного набора данных мы перемещаем окно ввода по тексту. Если шаг установлен на 1, мы сдвигаем окно ввода на 1 позицию при создании следующего пакета. Если мы установим шаг равным размеру окна ввода, мы сможем предотвратить перекрытие между пакетами.

![](images/llm2.14.png)


Упражнение 2.2. Загрузчики данных с разными шагами и размерами контекста.

Чтобы лучше понять, как работает загрузчик данных, попробуйте запустить его с различными настройками, такими как max_length=2 и шаг=2 и max_length=8 и шаг=2.




Размеры пакетов, равные 1, например, которые мы до сих пор выбирали из загрузчика данных, полезны для иллюстративных целей. Если у вас есть предыдущий опыт глубокого обучения, вы, возможно, знаете, что небольшие размеры пакетов требуют меньше памяти во время обучения, но приводят к более шумным обновлениям модели. Как и в обычном глубоком обучении, размер пакета — это компромисс и гиперпараметр, с которым можно экспериментировать при обучении LLM.

Прежде чем мы перейдем к двум последним разделам этой главы, посвященным созданию векторов внедрения из идентификаторов токенов, давайте кратко рассмотрим, как мы можем использовать загрузчик данных для выборки с размером пакета больше 1:

In [18]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4)
 
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[37733,  2346,   284,   884],
        [  764,   764,   764,   198],
        [  286,   511,   512,  1741],
        [ 1781,   314,  1422,   470],
        [  438, 14363, 10568,   852],
        [  502,    11,   290,   284],
        [  674,  1611,   286,  1242],
        [  766,   326,   314,  3521]])

Targets:
 tensor([[ 2346,   284,   884, 14177],
        [  764,   764,   198,   198],
        [  511,   512,  1741,    13],
        [  314,  1422,   470,  1560],
        [14363, 10568,   852,  5729],
        [   11,   290,   284,  3285],
        [ 1611,   286,  1242,   526],
        [  326,   314,  3521,   470]])


Обратите внимание, что мы увеличиваем шаг до 4. Это необходимо для полного использования набора данных (мы не пропускаем ни одного слова), но также для избежания любого перекрытия между пакетами, поскольку большее перекрытие может привести к увеличению переобучения.

В последних двух разделах этой главы мы реализуем уровни внедрения, которые преобразуют идентификаторы токенов в непрерывные векторные представления, которые служат форматом входных данных для LLM.


2.7 Creating token embeddings

The last step for preparing the input text for LLM training is to convert the token IDs into embedding vectors, as illustrated in Figure 2.15, which will be the focus of these two last remaining sections of this chapter.

Figure 2.15 Preparing the input text for an LLM involves tokenizing text, converting text tokens to token IDs, and converting token IDs into vector embedding vectors. In this section, we consider the token IDs created in previous sections to create the token embedding vectors.



In addition to the processes outlined in Figure 2.15, it is important to note that we initialize these embedding weights with random values as a preliminary step. This initialization serves as the starting point for the LLM's learning process. We will optimize the embedding weights as part of the LLM training in chapter 5.

A continuous vector representation, or embedding, is necessary since GPT-like LLMs are deep neural networks trained with the backpropagation algorithm. If you are unfamiliar with how neural networks are trained with backpropagation, please read section A.4, Automatic differentiation made easy, in Appendix A.

Let's illustrate how the token ID to embedding vector conversion works with a hands-on example. Suppose we have the following four input tokens with IDs 2, 3, 5, and 1:

input_ids = torch.tensor([2, 3, 5, 1])

For the sake of simplicity and illustration purposes, suppose we have a small vocabulary of only 6 words (instead of the 50,257 words in the BPE tokenizer vocabulary), and we want to create embeddings of size 3 (in GPT-3, the embedding size is 12,288 dimensions):

vocab_size = 6
output_dim = 3

Using the vocab_size and output_dim, we can instantiate an embedding layer in PyTorch, setting the random seed to 123 for reproducibility purposes:

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

The print statement in the preceding code example prints the embedding layer's underlying weight matrix:

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)



We can see that the weight matrix of the embedding layer contains small, random values. These values are optimized during LLM training as part of the LLM optimization itself, as we will see in upcoming chapters. Moreover, we can see that the weight matrix has six rows and three columns. There is one row for each of the six possible tokens in the vocabulary. And there is one column for each of the three embedding dimensions.

After we instantiated the embedding layer, let's now apply it to a token ID to obtain the embedding vector:

print(embedding_layer(torch.tensor([3])))

The returned embedding vector is as follows:

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)

If we compare the embedding vector for token ID 3 to the previous embedding matrix, we see that it is identical to the 4th row (Python starts with a zero index, so it's the row corresponding to index 3). In other words, the embedding layer is essentially a look-up operation that retrieves rows from the embedding layer's weight matrix via a token ID.


Embedding layers versus matrix multiplication

For those who are familiar with one-hot encoding, the embedding layer approach above is essentially just a more efficient way of implementing one-hot encoding followed by matrix multiplication in a fully connected layer, which is illustrated in the supplementary code on GitHub at https://github.com/rasbt/LLMs-from-scratch/tree/main/ch02/03_bonus_embedding-vs-matmul. Because the embedding layer is just a more efficient implementation equivalent to the one-hot encoding and matrix-multiplication approach, it can be seen as a neural network layer that can be optimized via backpropagation.

Previously, we have seen how to convert a single token ID into a three-dimensional embedding vector. Let's now apply that to all four input IDs we defined earlier (torch.tensor([2, 3, 5, 1])):

print(embedding_layer(input_ids))

The print output reveals that this results in a 4x3 matrix:

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)

Each row in this output matrix is obtained via a lookup operation from the embedding weight matrix, as illustrated in Figure 2.16.

Figure 2.16 Embedding layers perform a look-up operation, retrieving the embedding vector corresponding to the token ID from the embedding layer's weight matrix. For instance, the embedding vector of the token ID 5 is the sixth row of the embedding layer weight matrix (it is the sixth instead of the fifth row because Python starts counting at 0). For illustration purposes, we assume that the token IDs were produced by the small vocabulary we used in section 2.3.

This section covered how we create embedding vectors from token IDs. The next and final section of this chapter will add a small modification to these embedding vectors to encode positional information about a token within a text.

### 2.7 Создание вложений токенов

Последним шагом подготовки входного текста для обучения LLM является преобразование идентификаторов токенов в векторы внедрения, как показано на рисунке 2.15, который будет в центре внимания двух последних оставшихся разделов этой главы.

Рисунок 2.15. Подготовка входного текста для LLM включает токенизацию текста, преобразование текстовых токенов в идентификаторы токенов и преобразование идентификаторов токенов в векторы векторного внедрения. В этом разделе мы рассматриваем идентификаторы токенов, созданные в предыдущих разделах, для создания векторов внедрения токенов.

![](images/llm2.15.png)



В дополнение к процессам, показанным на рисунке 2.15, важно отметить, что мы инициализируем эти веса внедрения случайными значениями в качестве предварительного шага. Эта инициализация служит отправной точкой процесса обучения LLM. Мы оптимизируем веса внедрения в рамках обучения LLM в главе 5.

Непрерывное векторное представление или встраивание необходимо, поскольку GPT-подобные LLM представляют собой глубокие нейронные сети, обученные с помощью алгоритма обратного распространения ошибки. Если вы не знакомы с тем, как нейронные сети обучаются с помощью обратного распространения ошибки, прочтите раздел A.4 «Легкая автоматическая дифференциация» в Приложении A.

Давайте проиллюстрируем, как работает векторное преобразование идентификатора токена для внедрения, на практическом примере. Предположим, у нас есть следующие четыре входных токена с идентификаторами 2, 3, 5 и 1:

In [19]:
input_ids = torch.tensor([2, 3, 5, 1])

#For the sake of simplicity and illustration purposes, suppose we have a small vocabulary of only 6 words (instead of the 50,257 words in the BPE tokenizer vocabulary), and we want to create embeddings of size 3 (in GPT-3, the embedding size is 12,288 dimensions):

vocab_size = 6
output_dim = 3


Для простоты и иллюстрации предположим, что у нас есть небольшой словарь, состоящий всего из 6 слов (вместо 50 257 слов в словаре токенизатора BPE), и мы хотим создать вложения размером 3 (в GPT-3 размер встраивания составляет 12 288 измерений):

Используя vocab_size и output_dim, мы можем создать экземпляр слоя внедрения в PyTorch, установив случайное начальное значение 123 в целях воспроизводимости:

In [20]:
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


Мы видим, что весовая матрица слоя внедрения содержит небольшие случайные значения. Эти значения оптимизируются во время обучения LLM как часть самой оптимизации LLM, как мы увидим в следующих главах. Более того, мы видим, что матрица весов имеет шесть строк и три столбца. В словаре имеется одна строка для каждого из шести возможных токенов. И есть один столбец для каждого из трех измерений внедрения.

После того как мы создали экземпляр слоя внедрения, давайте теперь применим его к идентификатору токена, чтобы получить вектор внедрения:

In [21]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


Если мы сравним вектор внедрения для токена с идентификатором 3 с предыдущей матрицей внедрения, мы увидим, что он идентичен 4-й строке (Python начинается с нулевого индекса, поэтому эта строка соответствует индексу 3). Другими словами, уровень внедрения — это, по сути, операция поиска, которая извлекает строки из весовой матрицы слоя внедрения через идентификатор токена.


Встраивание слоев в сравнении с умножением матриц

Для тех, кто знаком с горячим кодированием, описанный выше подход уровня внедрения — это, по сути, просто более эффективный способ реализации горячего кодирования с последующим умножением матриц на полностью связном уровне, что проиллюстрировано в дополнительном коде на GitHub по адресу https. ://github.com/rasbt/LLMs-from-scratch/tree/main/ch02/03_bonus_embedding-vs-matmul. Поскольку уровень внедрения — это просто более эффективная реализация, эквивалентная подходу «горячего кодирования» и умножения матриц, его можно рассматривать как уровень нейронной сети, который можно оптимизировать с помощью обратного распространения ошибки.

Ранее мы видели, как преобразовать одиночный идентификатор токена в трехмерный вектор внедрения. Давайте теперь применим это ко всем четырем входным идентификаторам, которые мы определили ранее.

In [22]:
#(torch.tensor([2, 3, 5, 1])):

print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


Каждая строка в этой выходной матрице получается посредством операции поиска из матрицы весов внедрения, как показано на рисунке 2.16.

Рисунок 2.16. Уровни внедрения выполняют операцию поиска, извлекая вектор внедрения, соответствующий идентификатору токена, из весовой матрицы уровня внедрения. Например, вектор внедрения токена с идентификатором 5 — это шестая строка матрицы весов слоя внедрения (это шестая строка вместо пятой, поскольку Python начинает отсчет с 0). В целях иллюстрации мы предполагаем, что идентификаторы токенов были созданы с помощью небольшого словаря, который мы использовали в разделе 2.3.
![](images/llm2.16.png)


В этом разделе описано, как мы создаем векторы внедрения на основе идентификаторов токенов. В следующем и последнем разделе этой главы к этим векторам внедрения будет добавлена ​​небольшая модификация для кодирования позиционной информации о токене в тексте.

2.8 Кодирование позиций слов

В предыдущем разделе мы преобразовали идентификаторы токенов в непрерывное векторное представление, так называемые встраивания токенов. В принципе, это подходящий ввод для LLM. Однако незначительным недостатком LLM является то, что их механизм самообслуживания, который будет подробно описан в главе 3, не имеет представления о положении или порядке токенов внутри последовательности.

Принцип работы ранее представленного уровня внедрения заключается в том, что один и тот же идентификатор токена всегда отображается в одно и то же векторное представление, независимо от того, где идентификатор токена расположен во входной последовательности, как показано на рисунке 2.17.

Рисунок 2.17. Уровень внедрения преобразует идентификатор токена в одно и то же векторное представление независимо от того, где он расположен во входной последовательности. Например, идентификатор токена 5, независимо от того, находится ли он в первой или третьей позиции во входном векторе идентификатора токена, приведет к одному и тому же вектору внедрения.


![](images/llm2.17.png)



В принципе, детерминированное, независимое от позиции внедрение идентификатора токена полезно для целей воспроизводимости. Однако, поскольку механизм самообслуживания LLM сам по себе также не зависит от позиции, полезно вводить дополнительную информацию о местоположении в LLM.

Для достижения этой цели существуют две широкие категории вложений с учетом позиции: относительные позиционные вложения и абсолютные позиционные вложения.

Абсолютные позиционные вложения напрямую связаны с конкретными позициями в последовательности. Для каждой позиции во входной последовательности к внедрению токена добавляется уникальное вложение, чтобы передать его точное местоположение. Например, первый токен будет иметь определенное позиционное вложение, второй токен — другое отдельное вложение и т. д., как показано на рисунке 2.18.


Рисунок 2.18. Позиционные внедрения добавляются к вектору внедрения токена для создания входных внедрений для LLM. Позиционные векторы имеют ту же размерность, что и исходные вложения токенов. Внедрения токенов для простоты показаны со значением 1.


?????????

In [23]:
output_dim = 256
vocab_size = 50257
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

Используя указанный выше token_embedding_layer, если мы выбираем данные из загрузчика данных, мы встраиваем каждый токен в каждом пакете в 256-мерный вектор. Если у нас размер пакета 8 по четыре токена в каждом, результатом будет тензор 8 x 4 x 256.

Давайте сначала создадим экземпляр загрузчика данных из раздела 2.6 «Выборка данных со скользящим окном»:

In [24]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length, stride=max_length, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


Как мы видим, тензор идентификатора токена имеет размер 8x4, что означает, что пакет данных состоит из 8 образцов текста по 4 токена в каждом.

Давайте теперь воспользуемся слоем внедрения, чтобы встроить эти идентификаторы токенов в 256-мерные векторы:

In [25]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


Как мы можем сказать на основе выходных данных тензора размером 8x4x256, каждый идентификатор токена теперь внедрен как 256-мерный вектор.

Для подхода абсолютного внедрения модели GPT нам просто нужно создать еще один слой внедрения, который имеет то же измерение, что и token_embedding_layer:

In [26]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_lengthe, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

NameError: name 'context_lengthe' is not defined

Как показано в предыдущем примере кода, входными данными для pos_embeddings обычно является вектор-заполнитель torch.arange(context_length), который содержит последовательность чисел 0, 1,..., вплоть до максимальной входной длины — 1. context_length равна переменная, которая представляет поддерживаемый входной размер LLM. Здесь мы выбираем его аналогично максимальной длине входного текста. На практике входной текст может быть длиннее, чем поддерживаемая длина контекста, и в этом случае нам придется обрезать текст.

Как мы видим, тензор позиционного вложения состоит из четырех 256-мерных векторов. Теперь мы можем добавить их непосредственно во встраивания токенов, где PyTorch добавит 4x256-мерный тензор pos_embeddings к каждому 4x256-мерному тензору встраивания токенов в каждом из 8 пакетов:

In [ ]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

Созданные нами input_embeddings, как показано на рис. 2.19, представляют собой примеры встроенных входных данных, которые теперь могут обрабатываться основными модулями LLM, реализацию которых мы начнем в главе 3.

Рисунок 2.19. В рамках конвейера обработки ввода входной текст сначала разбивается на отдельные токены. Эти токены затем преобразуются в идентификаторы токенов с использованием словаря. Идентификаторы токенов преобразуются в векторы внедрения, к которым добавляются позиционные внедрения аналогичного размера, в результате чего входные внедрения используются в качестве входных данных для основных слоев LLM.
![](images/llm2.18.png)

2.9 Резюме

LLM требуют преобразования текстовых данных в числовые векторы, известные как внедрения, поскольку они не могут обрабатывать необработанный текст. Вложения преобразуют дискретные данные (например, слова или изображения) в непрерывные векторные пространства, делая их совместимыми с операциями нейронных сетей.
На первом этапе необработанный текст разбивается на токены, которые могут быть словами или символами. Затем токены преобразуются в целочисленные представления, называемые идентификаторами токенов.
Специальные токены, такие как <|unk|> и <|endoftext|>, можно добавлять для улучшения понимания модели и обработки различных контекстов, например неизвестных слов или обозначения границы между несвязанными текстами.
Токенизатор кодирования пар байтов (BPE), используемый для LLM, таких как GPT-2 и GPT-3, может эффективно обрабатывать неизвестные слова, разбивая их на подслова или отдельные символы.
Мы используем подход скользящего окна для токенизированных данных для генерации пар входных данных для обучения LLM.
Встраивание слоев в PyTorch выполняет функцию поиска, получая векторы, соответствующие идентификаторам токенов. Полученные векторы внедрения обеспечивают непрерывное представление токенов, что имеет решающее значение для обучения моделей глубокого обучения, таких как LLM.
Хотя встраивания токенов обеспечивают согласованные векторные представления для каждого токена, им не хватает понимания положения токена в последовательности. Чтобы исправить это, существуют два основных типа позиционных вложений: абсолютное и относительное. Модели OpenAI GPT используют абсолютные позиционные внедрения, которые добавляются к векторам внедрения токенов и оптимизируются во время обучения модели.